In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline
import seaborn as sns
import pandas as pd
import re
import json
sns.set_style('darkgrid')

In [ ]:
import sys
sys.path.append('../../pytpc')
import pytpc

# Processing ASTAR data into JSON files

The data from ASTAR are just in a whitespace-delimited table. It will be easier to load them for interpolation at runtime if they are instead in JSON files (or something else more machine-readable).

In [ ]:
def read_astar(fp):
    data = []
    for line in fp:
        elmts = line.strip().split()
        if any([re.match(r'\d*\.\d*E[+-]\d*', x) for x in elmts]):
            data.append([float(x) for x in elmts])
    return data

def make_gas_dict(molar, comp, dedx):
    d = {'molar_mass': molar,
         'composition': comp,
         'dedx': dedx}
    return d

## Helium data

This is for pure helium gas

In [ ]:
with open('../data/raw/helium_astar.txt') as f:
    he_dedx = read_astar(f)

he_gas_dict = make_gas_dict(4.002, {'he': 1.0}, he_dedx)
with open('../data/gases/helium.json', 'w') as f:
    json.dump(he_gas_dict, f, indent=2)

## CO2 data

This is for pure CO2 gas

In [ ]:
with open('../data/raw/co2_astar.txt') as f:
    co2_dedx = read_astar(f)

co2_gas_dict = make_gas_dict(44.01, {'co2': 1.0}, co2_dedx)
with open('../data/gases/carbon_dioxide.json', 'w') as f:
    json.dump(co2_gas_dict, f, indent=2)

# Generating data for mixtures of gases

The mixtures can be found from weighted averages.

In [ ]:
def density(pressure, molar_mass):
    """Find density in g/cm^3 from pressure in Torr 
    and molar mass in g/mol
    """
    return pressure / 760. * molar_mass / 24040.

## Helium-CO2 90/10

In [ ]:
with open('../data/gases/helium.json') as f:
    he_dict = json.load(f)
with open('../data/gases/carbon_dioxide.json') as f:
    co2_dict = json.load(f)

In [ ]:
hecomass = he_dict['molar_mass'] * 0.9 + co2_dict['molar_mass']
hedata = np.array(he_dict['dedx'])
codata = np.array(co2_dict['dedx'])

In [ ]:
hespl = InterpolatedUnivariateSpline(hedata[:, 0], hedata[:, 1])
cospl = InterpolatedUnivariateSpline(codata[:, 0], codata[:, 1])

In [ ]:
ens = hedata[:, 0]

In [ ]:
de1 = [hespl(e) * 0.9 + cospl(e) * 0.1 for e in ens]
de2 = [(hespl(e) * density(760, he_dict['molar_mass']) + cospl(e) * density(760, co2_dict['molar_mass']))
       / density(760, he_dict['molar_mass']*0.9 + co2_dict['molar_mass']*0.1) for e in ens]

In [ ]:
%%timeit
(hespl(3.4) * density(760, he_dict['molar_mass']) + cospl(3.4) * density(760, co2_dict['molar_mass'])) / density(760, he_dict['molar_mass']*0.9 + co2_dict['molar_mass']*0.1)

In [ ]:
plt.plot(ens, de1)
plt.plot(ens, de2)
plt.loglog()

In [ ]:
gas = pytpc.gases.HeliumGas(150.)

In [ ]:
%timeit gas.energy_loss(3.4, 4, 2)

In [ ]:
plt.plot(ens, [hespl(x)*0.9 + cospl(x)*0.1 for x in ens])
plt.loglog()

Here's a plot of each gas's contribution to the stopping power (in MeV/m, this time). This is for a total pressure of 760 torr. It appears that the partial pressure of CO2 (10% of the total gas pressure) contributes nearly as much as the helium does (the other 90%). This implies that just using helium gas in the simulation causes an error of about a factor of 2 in the energy loss.

In [ ]:
plt.plot(hedata.energy, hedata.dedx * density(760*0.9, hemass) * 100, label='ASTAR He')
plt.plot(codata.energy, codata.dedx * density(760*0.1, co2mass) * 100, label='ASTAR CO2')
plt.plot(hecodata.energy, hecodata.dedx * density(760, avgmass) * 100, label='Weighted Mean')
plt.loglog()
plt.legend()
plt.ylabel('Stopping Power [MeV/m]')
plt.xlabel('Energy [MeV]');

## Fitting the weighted average

First I'll take a subset of the data, since we don't need it to fit well above a certain energy.

In [ ]:
fitdata = hecodata.copy()
fitdata.index = fitdata.energy
fitdata.drop('energy', 1, inplace=True)
fitdata = fitdata.loc[0.01:20.0]

In [ ]:
fitdata.plot()

In [ ]:
from scipy.optimize import curve_fit
from numpy import exp

The fit function is below:

$$
    f(x) = \frac{a}{x^b} \frac{1}{c + d\,x^{-e}} + f \exp(-g (x-h)^2)
$$

This came from Wolfi's Fortran program.

In [ ]:
def f(en, a, b, c, d, e, f, g, h):
    return a*(1./en**b)*(1./(c+d/(en**e))) + f*exp(-g*(en-h)**2)

Guess some parameters

In [ ]:
plt.plot(fitdata.index, [f(x, *[300., 0.18, 0.8, 1, 1.5, 0.5, 300., 1.5]) for x in fitdata.index])
plt.plot(fitdata.index, fitdata.dedx, '.')
plt.loglog()

Perform the actual fit

In [ ]:
popt, pcov = curve_fit(f, fitdata.index.values, fitdata.dedx.values, p0=[300., 0.18, 0.8, 1, 1.5, 0.5, 300., 1.5])
plt.plot(hecodata.energy, [f(x, *popt) for x in hecodata.energy])
plt.plot(hecodata.energy, hecodata.dedx, '.')
plt.loglog()
plt.xlim(0.001, 20)

In [ ]:
popt  # These are the optimal parameters

## SRIM CO2 Data

I didn't use this in the fit, but it's here for future reference

In [ ]:
with open('/Users/josh/Documents/Data/GasData/srim-co2.txt') as f:
    in_header = True
    while in_header:
        litems = f.readline().strip().split()
        if all([re.match(r'-+', x) for x in litems]) and len(litems) == 6:
            in_header = False
    
    srimdat = []
    while True:
        litems = f.readline().strip().split()
        if len(litems) == 1 and re.match(r'-+', litems[0]):
            break
        en, en_u, dedx_elec, dedx_nuc, *junk = litems
        if en_u == 'keV':
            en = float(en) * 1e-3
        elif en_u == 'MeV':
            en = float(en)
        else:
            raise ValueError('energy units?')
        
        dedx_elec = float(dedx_elec) * 1000
        dedx_nuc = float(dedx_nuc) * 1000
        
        srimdat.append([en, dedx_elec + dedx_nuc])
    
    srimdat = pd.DataFrame(srimdat, columns=['energy', 'dedx'])

In [ ]:
plt.plot(codata.energy, codata.dedx, label='ASTAR CO2')
plt.plot(srimdat.energy, srimdat.dedx, label='SRIM CO2')
plt.plot(hedata.energy, hedata.dedx, label='ASTAR He')
plt.plot(codata.energy, co2stop(codata.energy), label='Brendle CO2 fit')
plt.plot(simdat.energy, simdat.dedx, 'k--', label='Simulation Data')
plt.loglog();
plt.legend();
plt.xlabel('Energy [MeV]')
plt.ylabel('Stopping Power [MeV/(g/cm^2)]');
plt.savefig('/Users/josh/Desktop/stop.pdf')

# Simulated range

Use this to check the results

In [ ]:
pt = pytpc.Particle(4, 2, energy_per_particle=6/4.)
heco = pytpc.gases.HeCO2Gas(153.)
he = pytpc.gases.HeliumGas(153.)
ef = np.array([0, 0, 15e3])
bf = np.zeros(3)

simres = pytpc.track(pt, he, ef, bf)

pt.energy = 6
pt.position = np.array((0, 0, 0))
simres2 = pytpc.track(pt, heco, ef, bf)

In [ ]:
plt.plot(simres['pos'][:, 2], simres['en']*pt.mass_num)
plt.plot(simres2['pos'][:, 2], simres2['en']*pt.mass_num)

In [ ]:
from scipy.interpolate import InterpolatedUnivariateSpline

In [ ]:
plt.plot(hecodata.energy, hecodata.dedx, '.', label='heco')
plt.plot(hecodata.energy, [spl(x) for x in hecodata.energy])
plt.loglog()
plt.xlim(0.001, 20)
plt.legend()

In [ ]:
spl = InterpolatedUnivariateSpline(hecodata.energy, hecodata.dedx)

In [ ]:
plt.plot([spl(x) for x in hecodata.energy])

In [ ]:
spl.get_coeffs()

In [ ]:
%timeit InterpolatedUnivariateSpline(hecodata.energy, hecodata.dedx)

In [ ]:
import pickle

In [ ]:
%%timeit
with open('/Users/josh/Desktop/spline.p', 'br') as f:
    pickle.load(f)

In [ ]:
import glob, os

In [ ]:
glob.glob('../data/gases/carbon_')

In [ ]:
os.path.abspath(__file__[0] + '/../data')

In [ ]:
newheco = pytpc.gases.InterpolatedGasMixture(150, ('helium', 1), (
)
heco = pytpc.gases.HeCO2Gas(150)
newhe = pytpc.gases.InterpolatedGas('helium', 150)
he = pytpc.gases.HeliumGas(150)

In [ ]:
ens = np.logspace(-3, 3, 100)
plt.plot(ens, [newhe.energy_loss(e, 4, 2) for e in ens])
plt.plot(ens, [newheco.energy_loss(e, 4, 2) for e in ens])
plt.loglog()

In [ ]:
he.density, newhe.density

In [ ]:
heco.pressure, newheco.pressure

In [ ]:
newheco.components[0][0].density + newheco.components[1][0].density

In [ ]:
newheco.density

In [ ]:
import os

In [ ]:
os.path.abspath(os.path.join(os.path.dirname(pytpc.gases.__file__), '..', 'data', 'gases', 'helium' + '.json'))